In [1]:
# 필요한 라이브러리 로드
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

# 시각화 라이브러리 설정
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

# RANDOM_SEED와 LABELS 설정
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]

2023-04-13 09:12:13.212140: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-13 09:12:13.276276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-13 09:12:13.558757: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-13 09:12:13.560856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 09:12:15.033638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
import pandas as pd

filename = './data/biz_log.csv'
df = pd.read_csv( filename, sep=', ', engine='python')
df.to_json('./data/biz_log.ndjson', orient='records', force_ascii=False, lines=True)
df.to_csv('./data/biz_log.json.csv', sep=',', index=False)

df.head()

,proc_date,proc_hour,proc_time,emp_cd,emp_nm,user_id,user_ip,dept_cd,dept_nm,org_cd,...,assu_org_cd,assu_org_nm,assu_hq_cd,assu_hq_nm,audit_no,grade,status,log_seq,sys_cd,log_agency_cd
0,20171101,1030.0,36.0,20150068.0,황혜련,20150068.0,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41596.0,AB01,HI
1,20171101,1030.0,35.0,20150068.0,황혜련,20150068.0,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41595.0,AB01,HI
2,20171101,1026.0,38.0,20150068.0,황혜련,20150068.0,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41594.0,AB01,HI
3,20171101,1026.0,36.0,20150068.0,황혜련,20150068.0,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41593.0,AB01,HI
4,20171101,1425.0,21.0,20162405.0,박아결,20162405.0,10.100.4.66,CH9,정보화지원부,CH0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41592.0,AB01,HI


In [3]:
df.shape

(7786, 82)

In [4]:
df.isna().sum()

proc_date           0
proc_hour           1
proc_time           1
emp_cd              1
emp_nm              1
                 ... 
grade            7786
status           7786
log_seq             1
sys_cd              1
log_agency_cd       1
Length: 82, dtype: int64

In [5]:
# remove null column
remove_columns = [key for value, key in zip(df.isna().sum(), df.columns) if value == df.shape[0]]

print( remove_columns )

df.drop(columns=remove_columns, inplace=True)
print(df.isna().sum())


['hq_nm', 'posit_gu', 'acc_org_cd', 'acc_org_nm', 'acc_hq_cd', 'acc_hq_nm', 'bran_cd', 'bran_nm', 'nh_dept_cd', 'nh_dept_nm', 'cd_org_cd', 'cd_org_nm', 'car_cd', 'site_cd', 'member_div', 'ssn_name', 'sear_cont', 'cert_num', 'sear_log24', 'esta_sym', 'firm_nm', 'assu_ssn', 'assu_nm', 'contact_condition', 'psnl_inf_qry_hed_colnm', 'psnl_inf_cnts', 'output_pgm_id', 'inq_db', 'patient_cd', 'patient_nm', 'bloodno', 'result', 'scrn_id', 'scrn_nm', 'vcls', 'ssn_org_cd', 'ssn_org_nm', 'ssn_hq_cd', 'ssn_hq_nm', 'juri_out', 'firm_cd', 'firm_cd_nm', 'firm_hq_cd', 'firm_hq_nm', 'assu_org_cd', 'assu_org_nm', 'assu_hq_cd', 'assu_hq_nm', 'audit_no', 'grade', 'status']
proc_date            0
proc_hour            1
proc_time            1
emp_cd               1
emp_nm               1
user_id              1
user_ip              1
dept_cd              1
dept_nm              1
org_cd               1
org_nm               1
hq_cd                1
join_ssn             1
sear_val          5917
inq_seq         

In [6]:
name_columns = list(filter(lambda x: 'nm' in x, df.columns))
# 'prg_nm' ??
print(df[name_columns].isna().sum())
df.drop(columns=name_columns, inplace=True)
print(df.isna().sum())


emp_nm         1
dept_nm        1
org_nm         1
ykiho_nm    3100
busi_nm     7498
prg_nm       289
dtype: int64
proc_date            0
proc_hour            1
proc_time            1
emp_cd               1
user_id              1
user_ip              1
dept_cd              1
org_cd               1
hq_cd                1
join_ssn             1
sear_val          5917
inq_seq              1
per_inf_cd        5917
inq_reason        7341
ykiho_cd          1412
recv_no           5980
recv_yyyy         5980
busi_cd           7498
busi_dtl_contn    7674
button_cd            1
prg_id             289
req_url           7498
log_seq              1
sys_cd               1
log_agency_cd        1
dtype: int64


In [7]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{"host": "default-opensearch-node1-0c190-1595032-15144b153298.kr-gov.lb.naverncp.com", "port": 9200}],
    http_auth = ("admin", "admin"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)
client.info()

{'name': 'opensearch-node1',
 'cluster_name': 'opensearch-cluster',
 'cluster_uuid': 'BAa1QZeQQ0O06tlxAX0P8g',
 'version': {'distribution': 'opensearch',
  'number': '2.6.0',
  'build_type': 'tar',
  'build_hash': '7203a5af21a8a009aece1474446b437a3c674db6',
  'build_date': '2023-02-24T18:57:04.388618985Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [8]:
import json 

properties = dict()
for column in df.columns:
    properties.update( { column : { 'type' : 'text' } } )

properties

body = {
  "mappings": {
    "properties": properties
  }
}

print(json.dumps( body, indent=2))
# response = client.indices.create("biz_logs", body=body)

{
  "mappings": {
    "properties": {
      "proc_date": {
        "type": "text"
      },
      "proc_hour": {
        "type": "text"
      },
      "proc_time": {
        "type": "text"
      },
      "emp_cd": {
        "type": "text"
      },
      "user_id": {
        "type": "text"
      },
      "user_ip": {
        "type": "text"
      },
      "dept_cd": {
        "type": "text"
      },
      "org_cd": {
        "type": "text"
      },
      "hq_cd": {
        "type": "text"
      },
      "join_ssn": {
        "type": "text"
      },
      "sear_val": {
        "type": "text"
      },
      "inq_seq": {
        "type": "text"
      },
      "per_inf_cd": {
        "type": "text"
      },
      "inq_reason": {
        "type": "text"
      },
      "ykiho_cd": {
        "type": "text"
      },
      "recv_no": {
        "type": "text"
      },
      "recv_yyyy": {
        "type": "text"
      },
      "busi_cd": {
        "type": "text"
      },
      "busi_dtl_contn": {
     

In [9]:
from opensearchpy.helpers import bulk

bulk_data = []

for i, row in df.iterrows():
    body = dict()
    for key, value in row.items():
        body.update( { key : value } )

        bulk_data.append({
            "_index": "biz_logs",
            "_id": i,
            "_source": body
        })

     
    # print( body )

    # bulk(client, bulk_data)

print( bulk_data )